Note: this file is view only: you have to make a copy to change any of the code (using "file" in the top left of this notebook)
# User instruction for people who don't know code very well:
 - clicking on a cell makes it your "current cell", where you can the edit or run it.
 - pressing the "run" button (looks like a play button) on the top left of a cell will run the code in that cell. You can also do this with shift+enter at the same time.
 - you should run code in order (starting with the top cell) and avoid rerunning cells unless something goes wrong
 - You will need to import the .mp3 (audio only) of your desired youtube video into the file section on the leftmost tab in this google colab notebook (instructions for generating this are in a cell below)
 - some cells, when you run them, will prompt you to enter an input below the cell. Type the input into the little box that shows up below the cell and then press enter!

 # Other Notes (this is not perfect):
 - The audio model will not hear every word perfectly.
 - The sumarizing process will sometimes attritube quotes or actions to the wrong people in the video
 - the sumarizing process might miss key details if the summarizing chunks are too large (you can specify/change this to avoid that though).


# STEP 1 -- Import your youtube video mp3 file into the left side of the colab:

In [1]:
# if you need to convert audio from youtube, use
# https://yt1s.de/youtube-to-mp3?l=en (the website is kinda sus, so beware of granting browser permissions and stuff)

# define the name of the audio file (.mp3) (name is whatever the file name is once you've downloaded the .mp3)
# this example is just a random mrbeast video
audio_file = input('What is the name of the audio file being processed? (Include ".mp3" at the end): ')

What is the name of the audio file being processed? (Include ".mp3" at the end): mrbeast.mp3


# STEP 2 -- Other Setup

### Misc Imports and setup:
(this might take a minute or two. If you have already ran the full code all the way through with one video during this session, then you should be able to skip this one cell to save time)

In [2]:
# for most purposes, it's not worth understanding what's going on in this cell. Just run it and move on.
%%capture
!pip install git+https://github.com/openai/whisper.git
!pip install transformers
!pip install nltk gensim

# imports
import whisper
from transformers import pipeline
import gensim
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

# bring in the summarizing model
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")
audio_to_text_model = whisper.load_model("base")

### Turn the audio into text

In [3]:
# transcribing the mp3 into words using openAI whisper
result = audio_to_text_model.transcribe(audio_file)["text"]
print(result[:150])  #this line shows the first few words so you can check if it's being processed correctly

 I built a massive isolation chamber. And we're gonna see if these two strangers can survive in this chamber for the next 100 days. They have never me


### Print the Audio Transcript:

In [4]:
result

" I built a massive isolation chamber. And we're gonna see if these two strangers can survive in this chamber for the next 100 days. They have never met each other ever, Bailey. This is Susie. Susie, this is Bailey. Nice to meet you. Hi, nice to meet you. If the two of you can survive the next 100 days in here, I will give you the half a million dollars inside of this bowl. But if one of you leaves before the 100 days is up, you both get nothing. Alright, I think you guys understand the rules. Have fun. Okay, bye. This is gonna be crazy. Um... Yeah. This is actually like an insane asylum. They're currently looking at all the stuff we put in there. We gave them enough food for 100 days. Which is healthy, but basically the exact same thing over and over again. We also gave them their own private bathroom, which comes with the shower. And obviously has no cameras inside. And a bed to sleep on. They have everything they need to survive 100 days. It's just a question of, do they want it? I 

# STEP 3 -- First pass of summarizing

## Chunking text and summarizing it
Specify the chunk size after running the cells below (chunk size = ~number of words being summarized at once)
- Do not exceed ~750 for chunk size, since the summarizing AI model is too small to be able to process any more words than that at one time
- Chunk size = 1 means that the model will try to include only 1 word in each summarized section. This is not ideal.
- Good values probably sit somewhere between 300 and 700.
- Bigger values (close to 700) mean the summary will be a little smaller, which might be good for large videos.
- Alternatively, smaller values will allow more details and granularity in the summary.

#### Misc function definition (just run the cell, don't bother to read)

In [38]:
# defining a funciton that splits text into chunks of roughly the same size, for processing into the summarizer
def splitter(n, text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize variables
    chunks = []
    current_chunk = []
    current_chunk_word_count = 0

    # Group sentences into chunks approximately the size of 'n' words
    for sentence in sentences:
        sentence_word_count = len(sentence.split())

        if current_chunk_word_count + sentence_word_count > n and current_chunk:
            # If the current chunk is full, add it to the chunks list
            chunks.append(' '.join(current_chunk))
            current_chunk = []  # Reset the current chunk
            current_chunk_word_count = 0

        # Add the sentence to the chunk
        current_chunk.append(sentence)
        current_chunk_word_count += sentence_word_count

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

### Actually sumarizing:
(this might take a minute or two to run)
(Be sure to specify a chunk size after running)

In [46]:
chunk_size = input("Enter summarizer chunk size: ")
split_text = splitter(int(chunk_size), result)
first_pass = ''
i = 0
total_len = len(split_text)
# summarizing chunks of the audio and concatenating them together into the first_pass variable
if input("show what's being summarized as it get's processed? (y/n): ").lower() == 'y':
  print()
  for co, subtext in enumerate(split_text):
    print(f'Summarizing {co+1}/{total_len} Sections. Current text being summarized:  ', subtext, '\n\n'); i += 1
    first_pass += summarizer(subtext)[0].get('summary_text') + '\n'
else:
  print()
  for subtext in split_text:
    first_pass += summarizer(subtext)[0].get('summary_text') + '\n'

Enter summarizer chunk size: 300
show what's being summarized as it get's processed? (y/n): y

Summarizing 1/4 Sections. Current text being summarized:    I built a massive isolation chamber. And we're gonna see if these two strangers can survive in this chamber for the next 100 days. They have never met each other ever, Bailey. This is Susie. Susie, this is Bailey. Nice to meet you. Hi, nice to meet you. If the two of you can survive the next 100 days in here, I will give you the half a million dollars inside of this bowl. But if one of you leaves before the 100 days is up, you both get nothing. Alright, I think you guys understand the rules. Have fun. Okay, bye. This is gonna be crazy. Um... Yeah. This is actually like an insane asylum. They're currently looking at all the stuff we put in there. We gave them enough food for 100 days. Which is healthy, but basically the exact same thing over and over again. We also gave them their own private bathroom, which comes with the shower. And

# VIEW -- The (first) summary:

In [47]:
print(first_pass)

Susie and Bailey have never met each other before. Bailey built a massive isolation chamber to test if they can survive in it for the next 100 days. If they do, Bailey will give them half a million dollars. If one of them leaves before the 100 days is up, they both get nothing.
Suzy and Bailey are living inside a white cube. They have no idea what day or time it is. On day four they used the paper from their journals to make a deck of playing cards and on day five, they made a sign-in sheet. Bailey wrote the date and the ball.
Nolan got an interesting offer for the 10th day of the challenge. Every 10 days, he will get something that will make the challenge easier, but he has to pay for it with some of his prize money. Nolan can buy a private chef for $50,000 for 90 days.
Jimmy and his friends haven't seen the sun in 10 days and they haven't had any food for 10 days. They are tempted by the offer of $50,000 for the rest of the challenge, but they decide not to take it. There will be ano

# STEP 5 -- Summarize the Summary, if it is still too much to read:

In [48]:
# frankly this is not really necessary for the test audio file I included in the example

split_text2 = splitter(int(chunk_size), first_pass)
len2 = len(split_text2)
second_pass = ''
i = 0

if input("show what's being summarized as it get's processed? (y/n): ").lower() == 'y':
  for co, subtext in enumerate(split_text2):
    print(f'Sumarizing subtext {co+1}/{len2}: ', subtext, '\n\n'); i += 1
    second_pass += summarizer(subtext)[0].get('summary_text') + '\n'
else:
  for subtext in split_text2:
    second_pass += summarizer(subtext)[0].get('summary_text') + '\n'

print('Second Pass Final Summary:\n' + second_pass)

show what's being summarized as it get's processed? (y/n): y
Sumarizing subtext 1/1:  Susie and Bailey have never met each other before. Bailey built a massive isolation chamber to test if they can survive in it for the next 100 days. If they do, Bailey will give them half a million dollars. If one of them leaves before the 100 days is up, they both get nothing. Suzy and Bailey are living inside a white cube. They have no idea what day or time it is. On day four they used the paper from their journals to make a deck of playing cards and on day five, they made a sign-in sheet. Bailey wrote the date and the ball. Nolan got an interesting offer for the 10th day of the challenge. Every 10 days, he will get something that will make the challenge easier, but he has to pay for it with some of his prize money. Nolan can buy a private chef for $50,000 for 90 days. Jimmy and his friends haven't seen the sun in 10 days and they haven't had any food for 10 days. They are tempted by the offer of $5

# STEP 6 -- Save the Summaries as a text file

In [ ]:
# saving your beautiful summaries, if you'd like
file_head = "summary_file"

with open(file_head + '.txt', 'w') as file:
    file.write('First summary: \n' + first_pass + '\n\n\n Second Summary: \n'+ second_pass)